### 輔助 02_pytorch_dataloader 在Windows不能執行的問題，此篇結果是在linux執行得到的

In [2]:
import torch
import numpy as np
import PIL.Image as Image
import torchvision.transforms.functional as FT
from torchvision import transforms

import json
__author__ = "Chih-Sheng(Tommy) Huang"

mytransform=transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        ])

class MyDataset_CarBrandsImages(torch.utils.data.Dataset):
    '''
    Class to load the dataset
    '''
    def __init__(self,transforms):
        
        with open('./dataset/kaggle/CarBrandsImages/carbrand.json') as jsonfile:
            data_load = json.load(jsonfile)
        self.imList = data_load['imagepaths']
        self.labelList = data_load['labels']
        self.transforms=transforms
        print('number of total data:{}'.format(len(self.imList)))
    def __len__(self):
        return len(self.imList)

    def __getitem__(self, idx):
        '''
        :param idx: Index of the image file
        :return: returns the image and corresponding label file.
        '''
        image_name = self.imList[idx]
        label = self.labelList[idx]
        
        # read image with PIL module
        image = Image.open(image_name, mode='r')
        image = image.convert('RGB')
        
        # Convert PIL label image to torch.Tensor
        image = self.transforms(image)
        label = torch.tensor(label)
        return image, label

if __name__ == "__main__":
    mydataset = MyDataset_CarBrandsImages(transforms=mytransform)
    mydata_loader = torch.utils.data.DataLoader(mydataset, batch_size=10, num_workers=0)
    for data, target in mydata_loader:
        print(data.size())
        print(target.size())
        break 

number of total data:4597
torch.Size([10, 3, 224, 224])
torch.Size([10])


In [4]:
def my_collate(batch):
    data, target = list(), list()
    for b in batch:
        data.append(b[0])
        target.append(b[1])
    data = torch.stack(data,dim=0)
    target = torch.stack(target,dim=0)
    return data, target
mydataset = MyDataset_CarBrandsImages(transforms=mytransform)
mydata_loader = torch.utils.data.DataLoader(mydataset, batch_size=100, num_workers=3, collate_fn = my_collate)
for data, target in mydata_loader:
    print(target.size())
         

number of total data:4597
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([97])


 **這邊我們測試看看如果需要讀圖檔下，開不開workers差多少。**

In [5]:
import time
import numpy as np
import matplotlib.pyplot as plt

mydataset = MyDataset_CarBrandsImages(transforms=mytransform)

def fun_timetest(data_loader):
    start = time.time()
    count=0
    for data, target in data_loader:
        count+=1
    timeimplement = time.time()-start
    print("total iteration: {}, a forloop time for whole dataset within CPU: {}s".format(count, timeimplement) )
    return timeimplement

for i in [0, 2, 4, 8]:
    mydata_loader = torch.utils.data.DataLoader(mydataset, batch_size=24, num_workers=i)
    print("\n num_workers: {}".format(i))
    timeimplement = fun_timetest(mydata_loader)

number of total data:4597

 num_workers: 0
total iteration: 192, a forloop time for whole dataset within CPU: 6.560382127761841s

 num_workers: 2
total iteration: 192, a forloop time for whole dataset within CPU: 3.8272275924682617s

 num_workers: 4
total iteration: 192, a forloop time for whole dataset within CPU: 2.1363651752471924s

 num_workers: 8
total iteration: 192, a forloop time for whole dataset within CPU: 1.7824933528900146s
